<a href="https://colab.research.google.com/github/edubdias/Projeto_Imersao_IA_Alura_Google/blob/main/Desafio_Imersao_IA_Alura_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [16]:
#Configurações iniciais
from pathlib import Path
import hashlib
import google.generativeai as genai

from google.colab import userdata

GOOGLE_API_KEY=userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [17]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [18]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [19]:
model_name='models/gemini-1.5-pro-latest'
model = genai.GenerativeModel(model_name,
                              generation_config=generation_config,
                              safety_settings=safety_settings,)

In [20]:
from google.colab import files
import requests
import os

# Carregar imagens com os modelos de exercícios
image_files = [
  {
    'url': 'http://www.bynexti.com.br/crucifixo-inclinado.jpg',
    'name': 'crucifixo-inclinado.jpg'
  },
  {
    'url': 'http://www.bynexti.com.br/supino-inclinado-com-barra.jpg',
    'name': 'supino-inclinado-com-barra.jpg'
  },
  {
    "url": 'http://www.bynexti.com.br/supino-reto.jpg',
    "name": 'supino-reto.jpg'
  },
]

# Fazer download das imagens
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.9"
           }

for fn in image_files:
  # Remove imagem antes de fazer upload
  image_file = ('/content/' + fn['name'])

  if not os.path.isfile(image_file):
    image_data = requests.get(url=fn['url'], headers=headers)
    with open(image_file, 'wb') as handler:
      handler.write(image_data.content)

    print(f'Imagem {image_file} copiada...\n')

# Dados padrao do modelo
image_path1 = '/content/crucifixo-inclinado.jpg'
image_path2 = '/content/supino-inclinado-com-barra.jpg'
image_path3 = '/content/supino-reto.jpg'

prompt_parts = [
  "Imagem de exercício com a forma correta de executá-lo.",

  # Item 1
  "Imagem: ",
  genai.upload_file(image_path1),
  "Exercício: Crucifixo inclinado",
  "Postura: CORRETA",

  # Item 2
  "Imagem: ",
  genai.upload_file(image_path2),
  "Exercício: Supino inclinado com barra",
  "Postura: CORRETA",

  # Item 3
  "Imagem: ",
  genai.upload_file(image_path3),
  "Exercício: Supino reto",
  "Postura: CORRETA",
]

# Inicia chat
chat = model.start_chat(history=[])

# Inicia modelo de dados
response = chat.model.generate_content(prompt_parts)

# Capturar imagem para análise
print('Faça o upload da imagem do exercício a ser analisado.\nClique em Cancelar para abandonar.\n')

image_count = 0

# Permite 5 consultas
while (image_count < 6):
  image_count += 1

  # Fazer o upload da imagem
  uploaded = files.upload()

  # Caso não informado, abandona
  if (uploaded == {}):
    break

  uploaded_file_path = next(iter(uploaded))

  print('\n')

  # Subir para a IA
  prompt_parts = [
  ('Dada a imagem, identifique o exercício e avalie se a postura está CORRETA ou INCORRETA. Dê uma breve justificativa."'),
   "Imagem: ",
   genai.upload_file(uploaded_file_path),
   "Exercício: ",
   "Postura: ",
  ]

  response = chat.model.generate_content(prompt_parts)
  print(response.text, '\n')

  print('--------------------------------\n')

  print('Faça o upload da imagem do exercício a ser analisado.\nClique em Cancelar para abandonar.\n')

  # Remove as imagens para liberar espaço
  lsdi = os.listdir('/content')

  for fn in uploaded.keys():
    if fn in lsdi:
      os.remove(uploaded_file_path)

# Fim
print('\n\nObrigado por utilizar o GEMINI, a IA do Google...')

Faça o upload da imagem do exercício a ser analisado.
Clique em Cancelar para abandonar.



Saving crossover-na-polia-baixa.jpg to crossover-na-polia-baixa.jpg


Exercício: Elevação lateral com cabos

Postura: **INCORRETA**

Justificativa: A imagem demonstra uma inclinação do tronco para frente, o que pode indicar uma carga excessiva para o exercício. Além disso, a inclinação pode comprometer a ativação correta dos músculos do ombro, reduzindo a eficácia do exercício e aumentando o risco de lesões. 

--------------------------------

Faça o upload da imagem do exercício a ser analisado.
Clique em Cancelar para abandonar.





Obrigado por utilizar o GEMINI, a IA do Google...
